In [3]:
import pandas as pd
import numpy as np

RAW_CSV_URL = "https://raw.githubusercontent.com/tshewangla/bhutan-healthcare-ds-project/main/data/raw/health_indicators_btn.csv"

df_raw = pd.read_csv(RAW_CSV_URL)

print(df_raw.shape)
df_raw.head()


(19990, 17)


,GHO (CODE),GHO (DISPLAY),GHO (URL),YEAR (DISPLAY),STARTYEAR,ENDYEAR,REGION (CODE),REGION (DISPLAY),COUNTRY (CODE),COUNTRY (DISPLAY),DIMENSION (TYPE),DIMENSION (CODE),DIMENSION (NAME),Numeric,Value,Low,High
0,#indicator+code,#indicator+name,#indicator+url,#date+year,#date+year+start,#date+year+end,#region+code,#region+name,#country+code,#country+name,#dimension+type,#dimension+code,#dimension+name,#indicator+value+num,#indicator+value,#indicator+value+low,#indicator+value+high
1,PHE_HHAIR_POP_CATEGORY_FUELS,Population with primary reliance on fuels and ...,https://www.who.int/data/gho/data/indicators/i...,2012,2012,2012,SEAR,South-East Asia,BTN,Bhutan,RESIDENCEAREATYPE,RESIDENCEAREATYPE_TOTL,Total,0.0,0 [0 - 0.03],0.0,0.032994834
2,W12_imports_dutyfree_A,Health warnings on cigarette packaging law app...,https://www.who.int/data/gho/data/indicators/i...,2022,2022,2022,SEAR,South-East Asia,BTN,Bhutan,NaN,NaN,NaN,NaN,No,NaN,NaN
3,WHOSIS_000001,Life expectancy at birth (years),https://www.who.int/data/gho/data/indicators/i...,2002,2002,2002,SEAR,South-East Asia,BTN,Bhutan,SEX,SEX_BTSX,Both sexes,67.84566543,67.8 [67.1-68.6],67.08312422,68.55874474
4,NCD_CHOL_MEANNONHDL_A,"Mean Non-HDL cholesterol, age-standardized",https://www.who.int/data/gho/data/indicators/i...,1991,1991,1991,SEAR,South-East Asia,BTN,Bhutan,SEX,SEX_FMLE,Female,3.0,3.0 [2.7-3.3],2.7,3.3


In [4]:

df = df_raw[df_raw['GHO (CODE)'] != '#indicator+code'].copy()


df = df[df['COUNTRY (CODE)'] == 'BTN'].copy()

df_life = df[df['GHO (CODE)'] == 'WHOSIS_000001'].copy()

print(df_life['GHO (DISPLAY)'].unique())
print(df_life['DIMENSION (NAME)'].unique())
df_life.head()


['Life expectancy at birth (years)']
['Both sexes' 'Female' 'Male']


,GHO (CODE),GHO (DISPLAY),GHO (URL),YEAR (DISPLAY),STARTYEAR,ENDYEAR,REGION (CODE),REGION (DISPLAY),COUNTRY (CODE),COUNTRY (DISPLAY),DIMENSION (TYPE),DIMENSION (CODE),DIMENSION (NAME),Numeric,Value,Low,High
3,WHOSIS_000001,Life expectancy at birth (years),https://www.who.int/data/gho/data/indicators/i...,2002,2002,2002,SEAR,South-East Asia,BTN,Bhutan,SEX,SEX_BTSX,Both sexes,67.84566543,67.8 [67.1-68.6],67.08312422,68.55874474
129,WHOSIS_000001,Life expectancy at birth (years),https://www.who.int/data/gho/data/indicators/i...,2018,2018,2018,SEAR,South-East Asia,BTN,Bhutan,SEX,SEX_FMLE,Female,74.94852418,74.9 [74.3-75.6],74.30722839,75.63214889
628,WHOSIS_000001,Life expectancy at birth (years),https://www.who.int/data/gho/data/indicators/i...,2004,2004,2004,SEAR,South-East Asia,BTN,Bhutan,SEX,SEX_FMLE,Female,69.63696233,69.6 [69.1-70.3],69.09666521,70.3409005
637,WHOSIS_000001,Life expectancy at birth (years),https://www.who.int/data/gho/data/indicators/i...,2006,2006,2006,SEAR,South-East Asia,BTN,Bhutan,SEX,SEX_MLE,Male,69.17739464,69.2 [68.7-69.9],68.67982739,69.88847867
1178,WHOSIS_000001,Life expectancy at birth (years),https://www.who.int/data/gho/data/indicators/i...,2015,2015,2015,SEAR,South-East Asia,BTN,Bhutan,SEX,SEX_FMLE,Female,74.24073046,74.2 [73.7-75.0],73.7062613,74.96810485


In [5]:

cols_to_keep = [
    'YEAR (DISPLAY)',
    'COUNTRY (DISPLAY)',
    'DIMENSION (NAME)',
    'Numeric',
    'Low',
    'High'
]

df_life = df_life[cols_to_keep].copy()


df_life = df_life.rename(columns={
    'YEAR (DISPLAY)': 'year',
    'COUNTRY (DISPLAY)': 'country',
    'DIMENSION (NAME)': 'sex',
    'Numeric': 'life_expectancy',
    'Low': 'ci_low',
    'High': 'ci_high'
})


for col in ['life_expectancy', 'ci_low', 'ci_high']:
    df_life[col] = pd.to_numeric(df_life[col], errors='coerce')


df_life['year'] = pd.to_numeric(df_life['year'], errors='coerce').astype('Int64')


df_life = df_life.dropna(subset=['life_expectancy']).copy()


df_life = df_life.sort_values(['year', 'sex']).reset_index(drop=True)

df_life.head(10)


,year,country,sex,life_expectancy,ci_low,ci_high
0,2000,Bhutan,Both sexes,65.989561,65.331720,66.840308
1,2000,Bhutan,Female,66.272210,65.492444,67.087420
2,2000,Bhutan,Male,65.733433,65.095977,66.570549
3,2001,Bhutan,Both sexes,67.180309,66.594776,67.916677
4,2001,Bhutan,Female,67.663072,66.964688,68.370021
5,2001,Bhutan,Male,66.748684,66.017784,67.597398
6,2002,Bhutan,Both sexes,67.845665,67.083124,68.558745
7,2002,Bhutan,Female,68.427289,67.664294,69.070898
8,2002,Bhutan,Male,67.326824,66.584363,68.071776
9,2003,Bhutan,Both sexes,68.384536,67.766933,69.153985


In [6]:
print("Shape:", df_life.shape)
print(df_life['sex'].value_counts())
print(df_life['year'].min(), "→", df_life['year'].max())

df_life.describe()


Shape: (66, 6)
sex
Both sexes    22
Female        22
Male          22
Name: count, dtype: int64
2000 → 2021


,year,life_expectancy,ci_low,ci_high
count,66.0,66.000000,66.000000,66.000000
mean,2010.5,71.363622,70.742094,72.062175
std,6.392905,2.606439,2.624229,2.579809
min,2000.0,65.733433,65.095977,66.570549
25%,2005.0,69.435956,68.838191,70.116164
50%,2010.5,71.814607,71.217063,72.552234
75%,2016.0,73.281179,72.690825,73.954405
max,2021.0,75.939756,75.477979,76.664141


In [10]:
df_life.to_csv("life_expectancy_btn_clean.csv", index=False)


In [11]:
from google.colab import files
files.download("life_expectancy_btn_clean.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>